### To find the times where MMS is in a specific region
Because the MEC srvy data is wayyyy less per day than fgm burst

In [5]:
import pyspedas
import pytplot
import numpy as np
import xarray as xr
import h5py
import os
os.environ["MMS_DATA_DIR"] = "/tigress/kendrab/analysis-notebooks/mms_data/pydata/"

### Load rough location data, downloading if necessary

In [6]:
# state has smaller files but loads much slower
# so we are going with mec since space isn't a huge issue

pyspedas.mms.mec(trange=['2015-03-13', '2024-06-28'], varnames=['mms1_mec_r_gsm',])

Loading pydata/mms1/mec/srvy/l2/epht89q/2015/03/mms1_mec_srvy_l2_epht89q_20150313_v2.1.0.cdf
Loading pydata/mms1/mec/srvy/l2/epht89q/2015/03/mms1_mec_srvy_l2_epht89q_20150314_v2.1.0.cdf
Loading pydata/mms1/mec/srvy/l2/epht89q/2015/03/mms1_mec_srvy_l2_epht89q_20150315_v2.1.0.cdf
Loading pydata/mms1/mec/srvy/l2/epht89q/2015/03/mms1_mec_srvy_l2_epht89q_20150316_v2.1.0.cdf
Loading pydata/mms1/mec/srvy/l2/epht89q/2015/03/mms1_mec_srvy_l2_epht89q_20150317_v2.1.0.cdf
Loading pydata/mms1/mec/srvy/l2/epht89q/2015/03/mms1_mec_srvy_l2_epht89q_20150318_v2.1.0.cdf
Loading pydata/mms1/mec/srvy/l2/epht89q/2015/03/mms1_mec_srvy_l2_epht89q_20150319_v2.1.0.cdf
Loading pydata/mms1/mec/srvy/l2/epht89q/2015/03/mms1_mec_srvy_l2_epht89q_20150320_v2.1.0.cdf
Loading pydata/mms1/mec/srvy/l2/epht89q/2015/03/mms1_mec_srvy_l2_epht89q_20150321_v2.1.0.cdf
Loading pydata/mms1/mec/srvy/l2/epht89q/2015/03/mms1_mec_srvy_l2_epht89q_20150322_v2.1.0.cdf
Loading pydata/mms1/mec/srvy/l2/epht89q/2015/03/mms1_mec_srvy_l2_epht8

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Loaded variables:
mms1_mec_r_gsm


['mms1_mec_r_gsm']

### Select data where the coordinates are what we want them to be

In [7]:
# data is in kilometers so we need to include that 
re = 6378

In [8]:
def condition(pos_data):
    x_cond = np.logical_and(-30*re < pos_data[:,0], pos_data[:,0] < -10*re)
    y_cond = np.logical_and(-10*re < pos_data[:,1], pos_data[:,1] < 10*re)
    z_cond = np.logical_and(-5*re < pos_data[:,2], pos_data[:,2] < 5*re)
    cond = np.logical_and(np.logical_and(x_cond, y_cond), z_cond)
    return cond

In [9]:
pared_data = pytplot.data_quants['mms1_mec_r_gsm'].where(condition)  # returns nans where not true

28-Jun-24 09:38:51: /home/kendrab/.conda/envs/pyspedas/lib/python3.7/site-packages/xarray/core/pycompat.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  duck_array_version = LooseVersion("0.0.0")



### Compare pared and initial data to see if it makes sense

In [10]:
pared_data

<xarray.DataArray 'mms1_mec_r_gsm' (time: 9682562, v_dim: 3)>
array([[nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       ...,
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan]])
Coordinates:
  * time     (time) datetime64[ns] 2015-03-13 ... 2024-05-25T23:59:30
Dimensions without coordinates: v_dim
Attributes:
    CDF:           {'VATT': {'CATDESC': 'Geocentric Solar Magnetospheric (GSM...
    data_att:      {'coord_sys': 'GSM', 'units': 'km', 'depend_1_units': 'km'...
    plot_options:  {'xaxis_opt': {'axis_label': 'Time', 'crosshair': 'X', 'x_...

In [11]:
pytplot.data_quants['mms1_mec_r_gsm']

<xarray.DataArray 'mms1_mec_r_gsm' (time: 9682562, v_dim: 3)>
array([[            nan,             nan,             nan],
       [            nan,             nan,             nan],
       [            nan,             nan,             nan],
       ...,
       [-34498.02964239, -37557.79926561, -41045.91511896],
       [-34507.15605796, -37628.50824604, -41094.94179472],
       [-34516.23945269, -37699.21206973, -41143.87124205]])
Coordinates:
  * time     (time) datetime64[ns] 2015-03-13 ... 2024-05-25T23:59:30
Dimensions without coordinates: v_dim
Attributes:
    CDF:           {'VATT': {'CATDESC': 'Geocentric Solar Magnetospheric (GSM...
    data_att:      {'coord_sys': 'GSM', 'units': 'km', 'depend_1_units': 'km'...
    plot_options:  {'xaxis_opt': {'axis_label': 'Time', 'crosshair': 'X', 'x_...

### Find the start and stop times for each segment within the target area

In [12]:
good_slice_list = np.ma.clump_unmasked(np.ma.masked_invalid(pared_data[:,0])) # mask should only depend on axis 1 so this is ok

In [13]:
len(good_slice_list)

513

In [14]:
pared_data.time[good_slice_list[0]]

<xarray.DataArray 'time' (time: 265)>
array(['2015-04-05T05:57:00.000000000', '2015-04-05T05:57:30.000000000',
       '2015-04-05T05:58:00.000000000', ..., '2015-04-05T08:08:00.000000000',
       '2015-04-05T08:08:30.000000000', '2015-04-05T08:09:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2015-04-05T05:57:00 ... 2015-04-05T08:09:00

### Save each time interval to a file

In [15]:
with h5py.File('interval_times.h5','w') as file:
    interval_times = file.create_dataset('times', (len(good_slice_list), 2), dtype=h5py.special_dtype(vlen=bytes))
    for i, good_slice in enumerate(good_slice_list):
        timeslice = pared_data.time[good_slice]
        # use datetime.datetime to convert to pyspedas' preferred string format for inputs
        # need to convert to datetime64[s] because this doesn't work with nanosecond resolution
        start_str = timeslice[0].values.astype('datetime64[s]').item().strftime('%Y-%m-%d/%H:%M:%S')
        end_str = timeslice[-1].values.astype('datetime64[s]').item().strftime('%Y-%m-%d/%H:%M:%S')
        interval_times[i,0] = start_str.encode('ascii')
        interval_times[i,1] = end_str.encode('ascii')
    